## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe_connected'
%matplotlib inline

# Task 7 - Calculate Days On Hand for all SKUs

In [2]:
# Reading Data
stock_data = pd.read_excel("dataset2.xlsx", sheet_name='Stocks')
sales_data = pd.read_excel("dataset2.xlsx", sheet_name='Sales')

## 1. Getting the Total Number of Sales of each SKU During The 10 Days Period

In [3]:
total_sales_per_sku = pd.pivot_table(sales_data, 
                                     index='sku', 
                                     values='Sales_Count', 
                                     aggfunc='sum')
total_sales_per_sku

,Sales_Count
sku,
AA سوني زنك (شرنك) - 4 حجر,8
AA فيليبس (شرنك) - 4 حجر,20
AAAفيليبس الكالين كارت - 2 حجر,1
D فيليبس (شرنك) - 2 حجر,9
أرز الضحى مصرى - 5 كيلو,46
...,...
ويفر فانيليا بسكو مصر - 2 جنية,186
ويفر كتاكيتو مغطي بالشوكولاته - 5 جنية,221
ويفر كتاكيتو ميني - 3 جنية,207


## 2. Merging Stock and Sales data

In [4]:
# Set 'sku' as index & Adding Total Sold Items Per SKU
stock_data.set_index('sku', inplace=True)
stock_sales_data = total_sales_per_sku.merge(stock_data, how='left', left_index=True, right_index=True).reset_index()
stock_sales_data.drop_duplicates('sku', inplace=True)
stock_sales_data.rename(columns={'Sales_Count': 'total_sold_items'}, inplace=True)
stock_sales_data.reset_index(inplace=True, drop=True)
stock_sales_data

,sku,total_sold_items,Stock
0,AA سوني زنك (شرنك) - 4 حجر,8,78
1,AA فيليبس (شرنك) - 4 حجر,20,50
2,AAAفيليبس الكالين كارت - 2 حجر,1,15
3,D فيليبس (شرنك) - 2 حجر,9,87
4,أرز الضحى مصرى - 5 كيلو,46,114
...,...,...,...
892,ويفر فانيليا بسكو مصر - 2 جنية,186,253
893,ويفر كتاكيتو مغطي بالشوكولاته - 5 جنية,221,536
894,ويفر كتاكيتو ميني - 3 جنية,207,632
895,ويفر كوماندا بيور محشو بكريمه الشوكولاته - 2 جنية,1,26


## 3. Calculating Days On Hand For each SKU

In [5]:
# Calculate Days On Hand for each SKU
# Period = 10 Days
stock_sales_data['days_on_hand'] = np.floor(stock_sales_data['Stock'] / (stock_sales_data['total_sold_items'] / 10))
stock_sales_data

,sku,total_sold_items,Stock,days_on_hand
0,AA سوني زنك (شرنك) - 4 حجر,8,78,97.0
1,AA فيليبس (شرنك) - 4 حجر,20,50,25.0
2,AAAفيليبس الكالين كارت - 2 حجر,1,15,150.0
3,D فيليبس (شرنك) - 2 حجر,9,87,96.0
4,أرز الضحى مصرى - 5 كيلو,46,114,24.0
...,...,...,...,...
892,ويفر فانيليا بسكو مصر - 2 جنية,186,253,13.0
893,ويفر كتاكيتو مغطي بالشوكولاته - 5 جنية,221,536,24.0
894,ويفر كتاكيتو ميني - 3 جنية,207,632,30.0
895,ويفر كوماندا بيور محشو بكريمه الشوكولاته - 2 جنية,1,26,260.0


## 4. Visualizing Top and Lowest SKUs in terms of Days On Hand

In [6]:
# Get The Highest 10 Days On Hand Items
top_10_items = stock_sales_data.sort_values(by='days_on_hand', ascending=False).head(10)

# Plot the results
fig =  px.bar(data_frame= top_10_items,
              x='sku', 
              y='days_on_hand',
              color='days_on_hand',
              text='days_on_hand',
              color_continuous_scale='deep')

fig.update_xaxes(ticklabelposition="inside top", title='SKU')
fig.update_yaxes(ticklabelposition="inside top", title='Days On Hand')
fig.update_layout(font=dict(family='Arial', size=16), 
                  width=1400, 
                  height=900)
fig.show()

### 5- Items With Zero Days On Hand (Out Of Stock in 0 Days)

In [7]:
# Select Items that are going to be out of stock in 0 days
zero_days_on_hand = stock_sales_data[stock_sales_data['days_on_hand'] == 0]
zero_days_on_hand

,sku,total_sold_items,Stock,days_on_hand
139,تاوتاو كيك الحليب مغطي بالشوكولاتةالبيضاء - 2...,39,3,0.0
171,جبنة دومتي بلس بالزيتون - 500 جرام,67,5,0.0
218,حلوى هولز مينتول كبير 9 قطع - 5 جنية,15,1,0.0
239,دقيق الضحي - 1 كيلو,394,34,0.0
244,ذرة فيشار الضحى - 500 جرام,278,10,0.0
302,سائل اطباق بريل متكرر - 35 جرام,293,2,0.0
354,شاى ليبتون أخضر بدون مرارة (عرض) - 25 فتلة,164,3,0.0
362,شاي ليبتون اخضر بالنعناع (عرض) - 25 فتلة,279,1,0.0
394,صابون جو كريمي مع الصبار و الليمون - 125 جرام,40,1,0.0
395,صابون جو كريمي مع اللوز - 125 جرام,47,1,0.0


## Task 8 - Calculate Stocks To Be Purchased From Suppliers Per SKU to reach 7 Days On Hands

In [8]:
# Days on hand = Stocck / (total_sold_items / # of days)
# Stock = (total_sold_items / # of days) * 7
pd.set_option('mode.chained_assignment', None)
less_than_7_doh = stock_sales_data[stock_sales_data['days_on_hand'] < 7.0]
less_than_7_doh['demand_stock'] = np.ceil(( less_than_7_doh['total_sold_items'] / 10) * 7)
less_than_7_doh['to_purchase_items_count'] = less_than_7_doh['demand_stock'] - less_than_7_doh['Stock']
less_than_7_doh

,sku,total_sold_items,Stock,days_on_hand,demand_stock,to_purchase_items_count
9,أرز الضحى مصري - 1 كيلو,153,84,5.0,108.0,24.0
16,أرز رفيع الأهرام - 1 كيلو,50,25,5.0,35.0,10.0
29,اريال جل - 2.5 لتر,18,2,1.0,13.0,11.0
33,افيريدي ازرق AA - 20 حجر,37,16,4.0,26.0,10.0
34,افيريدي ازرق AAA - 40 حجر,35,16,4.0,25.0,9.0
...,...,...,...,...,...,...
867,نسكافية بندق 3*1 ظرف (عرض) - 18 جرام,97,9,0.0,68.0,59.0
868,نسكافية شوكولاتة 3*1 ظرف (عرض) - 18 جرام,99,52,5.0,70.0,18.0
869,نسكافية ظرف 3*1 (عرض) - 18 جرام,90,43,4.0,63.0,20.0
883,هوت صوص هاينز - 165 مل,81,39,4.0,57.0,18.0


## 2. Visualizing The Number of SKUs needed to be purchased to achieve 7 Days On Hand

In [9]:
# Plot the results
fig =  px.bar(data_frame= less_than_7_doh.sort_values('to_purchase_items_count').tail(50),
              y='sku', 
              x='to_purchase_items_count',
              color='to_purchase_items_count', 
              text='to_purchase_items_count',
              color_continuous_scale='deep', 
              orientation='h')

fig.update_xaxes(title='No of Items To Purchase')
fig.update_yaxes(title='SKU')
fig.update_layout(font=dict(family='Arial', size=16), 
                  width=1400, 
                  height=1200)

fig.write_image("Items_Purchases.png")
fig.show()